TO DO

Bring in Discharge date for Notes, 

SET UP LANGCHAIN

In [1]:
import pandas as pd
import numpy as np
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import os 
import json
from langchain_core.runnables import RunnablePassthrough

#define groq key 
groq_key = os.environ['GROQ_KEY']


In [2]:
#model variables
llama_70 = 'llama3-70b-8192'
llama_31 = 'llama-3.1-70b-versatile'
mixtral = "mixtral-8x7b-32768"
gemma_2 = 'gemma2-9b-it'
llama_tool_70 = 'llama3-groq-70b-8192-tool-use-preview'

gemma = ChatGroq(
    temperature=0,
    model=gemma_2,
    api_key=groq_key 
)

mixtral = ChatGroq(
    temperature=0,
    model=mixtral,
    api_key=groq_key 
)

llama3 = ChatGroq(
    temperature=0,
    model=llama_70,
    api_key=groq_key 
)

llama_tool = ChatGroq(
    temperature=0,
    model=llama_70,
    api_key=groq_key 
)

llama_3_1 = ChatGroq(
    temperature=0,
    model=llama_31,
    api_key=groq_key 
)

In [ ]:
system = "You are a knowledgeable medical provider who specializes in medication management."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | llama_3_1
chain.invoke({"text": "Explain what should be evaluated for PPI deprescribing for a patient at discharge."})

In [69]:
%pip install huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [80]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
embeddings = HuggingFaceEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
text_splitter = RecursiveCharacterTextSplitter()

GET LABELED EXAMPLES

In [6]:
labels = pd.read_csv("../Data/LabeledResponses.csv")

In [3]:
encounters = pd.read_csv('../Data/encounters.txt', delimiter='|')

In [8]:
encounters

,EncounterKey,PatientKey,Sex,Age_y,BirthDate,PtAdmitDate,PtDischargeDate,DRG,FinancialClass,AdmissionOrigin,AdmissionSource,AdmissionType,PrimaryDx,PresentOnAdmissionDiagnosisComboKey,HospitalAcquiredDiagnosisComboKey,DischargeDisposition,DischargePatientClass
0,D009F2D78A0374,DB8B8E7C227264,Male,69,1954-06-13,2023-09-01,2023-09-26,PATHOLOGICAL FRACTURES AND MUSCULOSKELETAL AND...,Medicare,ED Admission,Self Referred (Home),Emergency,Secondary malignant neoplasm of bone (CMS code),4723681,-1,Deceased,Inpatient
1,D00F3A8D5F43B2,D51024504C9BE5,Male,24,1998-09-12,2023-09-02,2023-09-30,OTHER DIGESTIVE SYSTEM AND ABDOMINAL PROCEDURES,Covered California Medi-Cal,Transfer Center Admission,Transfer - Outside Emer Dept,Urgent,Secondary malignant neoplasm of retroperitoneu...,111736,-1,Home or Self Care,Inpatient
2,D01FCB9EFFD6D5,D47F8C6663B2AC,Male,53,1970-07-23,2023-08-27,2023-09-06,PERCUTANEOUS AND OTHER INTRACARDIAC PROCEDURES...,Blue Shield,Transfer Center Admission,Clinic Referral,Urgent,Typical atrial flutter (CMS code),-1,-1,Home or Self Care,Inpatient
3,D02DC627A3743A,DDC17BF3C6B1D9,Male,87,1935-10-15,2023-09-20,2023-09-22,OTHER DIGESTIVE SYSTEM DIAGNOSES WITH CC,Medicare,ED Admission,Self Referred (Home),Emergency,"Vascular disorder of intestine, unspecified (C...",-1,-1,Home Health Care (Non UCSF),Inpatient
4,D03868A75BBE62,DAE2BFBDAFD04C,Female,59,1964-04-06,2023-08-26,2023-09-11,SEPTICEMIA OR SEVERE SEPSIS WITHOUT MV >96 HOU...,United Health Care,ED Admission,Self Referred (Home),Emergency,Other specified sepsis (CMS code),-1,-1,Home or Self Care,Inpatient
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,DFE758059BFA6F,DEE66864FC153C,Female,73,1950-07-16,2023-09-04,2023-09-06,REVISION OF HIP OR KNEE REPLACEMENT WITH CC,Medicare,OR Admission,Self Referred (Home),Routine/Elective,"Dislocation of internal right hip prosthesis, ...",-1,-1,Home Health Care (Non UCSF),Inpatient
518,DFE8D15882619F,D18606733E6081,Female,83,1939-10-14,2023-09-02,2023-09-08,OTHER CIRCULATORY SYSTEM DIAGNOSES WITH CC,Medicare Advantage HMO/Senior,Transfer Center Admission,Transfer - Outside Emer Dept,Urgent,Other specified complication of cardiac prosth...,26937919,-1,Home or Self Care,Inpatient
519,DFEB858A646876,DADAA3903E2F88,Male,87,1935-11-30,2023-09-17,2023-09-18,SIGNS AND SYMPTOMS WITHOUT MCC,Medicare,ED Admission,Self Referred (Home),Emergency,Neoplasm related pain (acute) (chronic),-1,-1,Skilled Nursing Facility,Inpatient
520,DFEE0F5F83A71E,DE27C1A4857799,Male,81,1941-11-03,2023-09-17,2023-09-19,MAJOR SMALL AND LARGE BOWEL PROCEDURES WITH CC,Medicare,OR Admission,Physician Referral,Routine/Elective,Encounter for attention to ileostomy (CMS code),-1,-1,Home or Self Care,Inpatient


In [9]:
labels

,key,reason,recommendation
0,D6253A5CE371EA,example with gastrointestinal bleed should con...,continue
1,DFAFED1811B871,example with a documented history of a GI blee...,continue
2,D4865B8BBB294E,example with GI bleeding that occurred during ...,continue
3,D00F3A8D5F43B2,example with symptoms of heartburn (upper GI s...,deprescribe
4,D8017C77BA15FA,14d h pylori treatment if pt treated for 2wks ...,stop
5,DA15CCF42ACF8B,"Barrett's esophagus, continue PPI",continue
6,D45728A2EFD315,"example with esophagitis, continue PPI",continue
7,DFB07E6B8F0957,example with chronic NSAID use and pharmacist ...,stop
8,D1BD3665C06499,example where pt should be on due to history o...,continue
9,D068E26FFF9F43,example where pt came in on and no clear reaso...,stop


In [11]:
df = encounters.merge(labels, left_on='EncounterKey', right_on='key')

In [96]:
notes = pd.read_csv('../Data/noteText.csv')
notes

,deid_note_key,EncounterKey,NoteDate,DepartmentSpecialty,ProviderSpecialty,ProviderType,NoteText
0,D00010F0E618AD,D304C167A23716,2023-09-15,Inpatient Nursing,NaN,Occupational Therapist,OCCUPATIONAL THERAPY CANCELED SESSION NOTE ...
1,D00015E35CE5B7,DD27EC809E2A28,2023-08-25,Inpatient Nursing,UCSF,Resident,History of cardiomyopathy presumed secondary t...
2,D000253FB63C7E,D054778440C77A,2023-09-10,Inpatient Nursing,General Surgery,Physician,Procedure(s) (LRB): KIDNEY TRANSPLANT CADAVER...
3,D0004A42F6443B,D8277269D1C037,2023-08-31,Inpatient Nursing,UCSF,Resident,NEUROSURGERY PROGRESS NOTE Hospital Course...
4,D0005D9BF5CAF1,D5A083F15D8290,2023-09-09,Inpatient Nursing,Nephrology,Physician,TRANSPLANT NEPHROLOGY PROGRESS NOTE Date of...
...,...,...,...,...,...,...,...
31211,DFFF6B55F41A96,D8757A00B31B22,2023-08-13,Inpatient Nursing,NaN,Physical Therapist,PHYSICAL THERAPY TREATMENT NOTE PT relevant...
31212,DFFF8CBF734480,D70969A027A062,2023-06-23,Inpatient Nursing,NaN,Registered Nurse,Mechanical Circulatory Support Indicatio...
31213,DFFF9A013C9A3D,DC5430B22637DD,2023-08-30,Inpatient Nursing,NaN,Pharmacist,ANTIMICROBIAL STEWARDSHIP CONSULT This p...
31214,DFFFB1BAD49145,D70969A027A062,2023-07-22,Inpatient Nursing,UCSF,Resident,Cardiac Surgery Lung Transplant Progress Note ...


In [48]:
#notes[notes.EncounterKey == 'D009F2D78A0374']

,deid_note_key,EncounterKey,NoteDate,DepartmentSpecialty,ProviderSpecialty,ProviderType,NoteText,key


In [97]:
notes = notes.merge(labels[['key']], left_on='EncounterKey', right_on='key')

In [98]:
selected_providers = ['Resident', 'Physician', 'Registered Nurse', 'Nurse Practitioner',
                      'Physician Assistant', 'Pharmacist', 'Licensed Vocational Nurse', 
                      'Medical Student', 'Pharmacy Student', 'Nursing Student', 
                      'Registered Dietitian', 'Dietetic Intern']
notes = notes[(notes['ProviderType'].isin(selected_providers)) | (notes['ProviderType'].isnull())]

In [99]:
notes

,deid_note_key,EncounterKey,NoteDate,DepartmentSpecialty,ProviderSpecialty,ProviderType,NoteText,key
0,D00061A8A25CE6,D8017C77BA15FA,2023-07-09,Inpatient Nursing,Nephrology,Physician,NEPHROLOGY FOLLOW UP CONSULT NOTE Events ...,D8017C77BA15FA
1,D00243904093A0,D8017C77BA15FA,2023-07-19,Inpatient Nursing,Neurology,Physician Assistant,This is an independent service. The availa...,D8017C77BA15FA
2,D004767E21ABBC,D8017C77BA15FA,2023-07-26,Inpatient Nursing,NaN,Nurse Practitioner,UCSF Medical Intensive Care Unit Progress No...,D8017C77BA15FA
3,D005AD8F471521,D45728A2EFD315,2023-08-31,Inpatient Nursing,UCSF,Resident,Gastroenterology Initial Consultation Note ...,D45728A2EFD315
4,D00B2936D4CF40,D0E44FD0BBD96F,2023-09-18,Inpatient Nursing,NaN,Registered Nurse,Problem: Pain Acute / Chronic- Adult Goal:...,D0E44FD0BBD96F
...,...,...,...,...,...,...,...,...
1636,DFF89641837FF7,D45728A2EFD315,2023-08-26,Inpatient Nursing,Critical Care Medicine,Nurse Practitioner,ADVANCED LUNG DISEASE PROGRESS NOTE This is ...,D45728A2EFD315
1637,DFFA2D7C1B2202,D4865B8BBB294E,2023-09-08,Inpatient Nursing,NaN,Nurse Practitioner,The following orders were created for panel or...,D4865B8BBB294E
1638,DFFBAEF40F860B,D45728A2EFD315,2023-08-19,Inpatient Nursing,NaN,Registered Nurse,Problem: Discharge Planning - Adult Goal: K...,D45728A2EFD315
1639,DFFC5634F74117,D45728A2EFD315,2023-09-16,Inpatient Nursing,NaN,Registered Nurse,Problem: Discharge Planning - Adult Goal: K...,D45728A2EFD315


In [100]:
import re
# Function to extract surrounding words
def extract_surrounding_text(text):
    match = re.search(r'(\S+\s+){0,6}DISCHARGE(\s+\S+){0,6}', text)
    if match:
        print(match.group(0))
    return match.group(0) if match else None

# Apply the function to the text column
notes['discharge_text'] = notes['NoteText'].apply(extract_surrounding_text)

#REMOVED these Notes

***** ***** DISCHARGE NOTE       ***** Surgery    Patient: ***** *****
UCSF MEDICAL CENTER - DISCHARGE SUMMARY     Patient Name: ***** ***** *****
UCSF MEDICAL CENTER - DISCHARGE SUMMARY - ATTENDING ONLY     Patient Name:
UCSF MEDICAL CENTER - DISCHARGE SUMMARY     Patient Name: ***** *****  *****
UCSF MEDICAL CENTER - DISCHARGE SUMMARY     Patient Name: ***** *****  *****
UCSF ORTHOPEDIC-SPINE SERVICE  PATIENT DISCHARGE INSTRUCTIONS    Name ***** ***** ***** *****
UCSF MEDICAL CENTER - DISCHARGE SUMMARY - ATTENDING ONLY     Patient Name:
UCSF MEDICAL CENTER - DISCHARGE SUMMARY - ATTENDING ONLY     Patient Name:
up chest pain per usual pathway     DISCHARGE PLANNING  Clinic appointments:   Clinic appointment: In
UCSF MEDICAL CENTER - DISCHARGE SUMMARY - ATTENDING ONLY     Patient Name:
***** ***** DISCHARGE NOTE       ***** Transplant    Date of Transplant:
UCSF MEDICAL CENTER - DISCHARGE SUMMARY     Patient Name: ***** *****  *****
UCSF MEDICAL CENTER - DISCHARGE SUMMARY     Patient N

In [101]:
notes[~notes.discharge_text.isna()]

,deid_note_key,EncounterKey,NoteDate,DepartmentSpecialty,ProviderSpecialty,ProviderType,NoteText,key,discharge_text
42,D04FF7732B6600,D068E26FFF9F43,2023-09-20,Inpatient Nursing,NaN,Pharmacist,***** ***** DISCHARGE NOTE ***** Surge...,D068E26FFF9F43,***** ***** DISCHARGE NOTE ***** Surgery...
128,D0F96CC39085E6,D45728A2EFD315,2023-09-17,Inpatient Nursing,NaN,Nurse Practitioner,UCSF MEDICAL CENTER - DISCHARGE SUMMARY Pa...,D45728A2EFD315,UCSF MEDICAL CENTER - DISCHARGE SUMMARY Pa...
167,D141F7560B4A73,DFB07E6B8F0957,2023-09-07,Inpatient Nursing,Hospital Medicine,Physician,UCSF MEDICAL CENTER - DISCHARGE SUMMARY - AT...,DFB07E6B8F0957,UCSF MEDICAL CENTER - DISCHARGE SUMMARY - ATTE...
244,D1FD093A7EF8D1,D17AFAD1D7BB68,2023-09-17,Inpatient Nursing,UCSF,Resident,UCSF MEDICAL CENTER - DISCHARGE SUMMARY ...,D17AFAD1D7BB68,UCSF MEDICAL CENTER - DISCHARGE SUMMARY Pa...
367,D36A386789BCB6,D5A7045ED60A2E,2023-09-22,Inpatient Nursing,Nurse Practitioner,Nurse Practitioner,UCSF MEDICAL CENTER - DISCHARGE SUMMARY ...,D5A7045ED60A2E,UCSF MEDICAL CENTER - DISCHARGE SUMMARY Pa...
429,D4124945C9DD11,D068E26FFF9F43,2023-09-19,Inpatient Nursing,NaN,Physician Assistant,UCSF ORTHOPEDIC-SPINE SERVICE PATIENT DISCHAR...,D068E26FFF9F43,UCSF ORTHOPEDIC-SPINE SERVICE PATIENT DISCHAR...
439,D4295B888B30AD,DFAFED1811B871,2023-09-05,Inpatient Nursing,Hospital Medicine,Physician,UCSF MEDICAL CENTER - DISCHARGE SUMMARY - AT...,DFAFED1811B871,UCSF MEDICAL CENTER - DISCHARGE SUMMARY - ATTE...
505,D4D6F1790BD9E0,D4BAEF91CA7588,2023-09-21,Inpatient Nursing,Hospital Medicine,Physician,UCSF MEDICAL CENTER - DISCHARGE SUMMARY - AT...,D4BAEF91CA7588,UCSF MEDICAL CENTER - DISCHARGE SUMMARY - ATTE...
620,D6077B944990DC,D00F3A8D5F43B2,2023-09-27,Inpatient Nursing,Family Medicine,Nurse Practitioner,This is an independent service. The availabl...,D00F3A8D5F43B2,up chest pain per usual pathway DISCHARGE ...
709,D6CC1BFAC9EE72,D6253A5CE371EA,2023-09-21,Inpatient Nursing,Hospital Medicine,Physician,UCSF MEDICAL CENTER - DISCHARGE SUMMARY - AT...,D6253A5CE371EA,UCSF MEDICAL CENTER - DISCHARGE SUMMARY - ATTE...


In [55]:
notes = notes[notes.discharge_text.isna()]
notes

,deid_note_key,EncounterKey,NoteDate,DepartmentSpecialty,ProviderSpecialty,ProviderType,NoteText,key,discharge_text
0,D00061A8A25CE6,D8017C77BA15FA,2023-07-09,Inpatient Nursing,Nephrology,Physician,NEPHROLOGY FOLLOW UP CONSULT NOTE Events ...,D8017C77BA15FA,None
1,D00243904093A0,D8017C77BA15FA,2023-07-19,Inpatient Nursing,Neurology,Physician Assistant,This is an independent service. The availa...,D8017C77BA15FA,None
2,D004767E21ABBC,D8017C77BA15FA,2023-07-26,Inpatient Nursing,NaN,Nurse Practitioner,UCSF Medical Intensive Care Unit Progress No...,D8017C77BA15FA,None
3,D005AD8F471521,D45728A2EFD315,2023-08-31,Inpatient Nursing,UCSF,Resident,Gastroenterology Initial Consultation Note ...,D45728A2EFD315,None
4,D00B2936D4CF40,D0E44FD0BBD96F,2023-09-18,Inpatient Nursing,NaN,Registered Nurse,Problem: Pain Acute / Chronic- Adult Goal:...,D0E44FD0BBD96F,None
...,...,...,...,...,...,...,...,...,...
1636,DFF89641837FF7,D45728A2EFD315,2023-08-26,Inpatient Nursing,Critical Care Medicine,Nurse Practitioner,ADVANCED LUNG DISEASE PROGRESS NOTE This is ...,D45728A2EFD315,None
1637,DFFA2D7C1B2202,D4865B8BBB294E,2023-09-08,Inpatient Nursing,NaN,Nurse Practitioner,The following orders were created for panel or...,D4865B8BBB294E,None
1638,DFFBAEF40F860B,D45728A2EFD315,2023-08-19,Inpatient Nursing,NaN,Registered Nurse,Problem: Discharge Planning - Adult Goal: K...,D45728A2EFD315,None
1639,DFFC5634F74117,D45728A2EFD315,2023-09-16,Inpatient Nursing,NaN,Registered Nurse,Problem: Discharge Planning - Adult Goal: K...,D45728A2EFD315,None


In [58]:
for i,row in df.iterrows():
    encounter_notes = notes[notes.EncounterKey == row['EncounterKey']]
    print(encounter_notes)
    break;

       deid_note_key    EncounterKey    NoteDate DepartmentSpecialty  \
13    D01721C3A10604  D00F3A8D5F43B2  2023-09-03   Inpatient Nursing   
17    D01BFD9C31A2DC  D00F3A8D5F43B2  2023-09-20   Inpatient Nursing   
20    D01FDFADE07C0E  D00F3A8D5F43B2  2023-09-30   Inpatient Nursing   
29    D0359C9D6E2599  D00F3A8D5F43B2  2023-09-17   Inpatient Nursing   
38    D04ABD517A07B5  D00F3A8D5F43B2  2023-09-27   Inpatient Nursing   
...              ...             ...         ...                 ...   
1585  DF8D83D6C33067  D00F3A8D5F43B2  2023-09-12   Inpatient Nursing   
1587  DF8F0727821584  D00F3A8D5F43B2  2023-09-17   Inpatient Nursing   
1616  DFD428E54333EB  D00F3A8D5F43B2  2023-09-19   Inpatient Nursing   
1631  DFF0C1C0DF264D  D00F3A8D5F43B2  2023-09-03   Inpatient Nursing   
1633  DFF52F126ED943  D00F3A8D5F43B2  2023-09-22   Inpatient Nursing   

            ProviderSpecialty        ProviderType  \
13          Hospital Medicine           Physician   
17                       UCSF

In [59]:
one_encounter = notes[notes.EncounterKey == 'D00F3A8D5F43B2']

In [81]:
from langchain_community.document_loaders import DataFrameLoader
loader = DataFrameLoader(
 data_frame = one_encounter,
 page_content_column = "NoteText",
 engine="pandas",
)

documents = loader.load_and_split()

In [82]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS


vector_store = FAISS.from_documents(documents, embeddings)

In [83]:
query = """ Based on the information from the note context, does the patient have any of the following:
              1. Mild to moderate esoophagitis
              2. GERD 
              3. Peptic Ulcer Disease
              4. Upper GI symptoms
              5. ICU Stress Ulcer Prophylaxis
              6. Barrett's Esophagus
              7. Chronic NSAID use with bleeding risk
              8. Severe esophagitis
              9. Documented history of bleeding GI ulcer """

In [87]:
df[df.EncounterKey == 'D00F3A8D5F43B2'].reason.iloc[0]

'example with symptoms of heartburn (upper GI symptoms) evaluate for endoscopy to determine if PPI is needed'

In [94]:
vector_store.similarity_search_with_score(query, search_type="similarity", k=5)[0]['']

(Document(metadata={'deid_note_key': 'D1B064B93CF136', 'EncounterKey': 'D00F3A8D5F43B2', 'NoteDate': '2023-09-12', 'DepartmentSpecialty': 'Inpatient Nursing', 'ProviderSpecialty': 'Hematology and Oncology', 'ProviderType': 'Nurse Practitioner', 'key': 'D00F3A8D5F43B2', 'discharge_text': None}, page_content='Q12H SCH    electrolyte-A bolus  1000 mL Intravenous Once    enoxaparin  1 mg/kg (Dosing Weight) Subcutaneous Q12H SCH    lidocaine  1 patch Topical Daily SCH    melatonin  10 mg Oral Daily At ***** *****    polyethylene glycol  17 g Oral BID    senna  17.2 mg Oral BID SCH     Continuous Infusions:  PRN Meds:   0.9% sodium chloride flush  10-20 mL Intravenous PRN    0.9% sodium chloride flush  3 mL Intravenous PRN    0.9% sodium chloride flush  30 mL Intravenous Once PRN    acetaminophen  1000 mg Oral Q8H PRN    ALPRAZolam  0.5 mg Oral Bedtime PRN    bisacodyL  10 mg Rectal Daily PRN    famotidine  20 mg Oral Daily PRN    HYDROmorphone  0.5 mg Intravenous Q3H PRN    hydrOXYzine  10 

In [88]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retriever = vector_store.as_retriever(search_type="similarity", k=5)

In [89]:
system = "You are a knowledgeable medical provider who specializes in medication management. Given a list of diagnosis and some snippets from patients notes {context}, answer if the patient notes contain any of the diagnosis."
prompt = ChatPromptTemplate.from_messages(
        [
        ("system", system), ("human", "{input}")
        ])
prompt.pretty_print()

================================ System Message ================================

You are a knowledgeable medical provider who specializes in medication management. Given a list of diagnosis and some snippets from patients notes {context}, answer if the patient notes contain any of the diagnosis.

================================ Human Message =================================

{input}


In [90]:
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [91]:
rag_chain = (
        {"context": retriever | format_docs,
         "input": RunnablePassthrough()}
        | prompt 
        | llama_3_1
        | StrOutputParser()
)


In [92]:
rag_chain = (
        {"context": retriever | format_docs,
         "input": RunnablePassthrough()}
        | prompt 
        | llama_3_1
        | StrOutputParser()
)


In [93]:
response = rag_chain.invoke("""Based on the information from the note context, does the patient have any of the following:
              1. Mild to moderate esoophagitis
              2. GERD 
              3. Peptic Ulcer Disease
              4. Upper GI symptoms
              5. ICU Stress Ulcer Prophylaxis
              6. Barrett's Esophagus
              7. Chronic NSAID use with bleeding risk
              8. Severe esophagitis
              9. Documented history of bleeding GI ulcer
              10. Explain the reasoning for your answer
            Return the answer for each of these as a formatted JSON object with the key being the condition and the value being a boolean value for the first 9.  For the final question, return a string with the reasoning for your answer.""")
print(response)

```json
{
  "Mild to moderate esophagitis": false,
  "GERD": false,
  "Peptic Ulcer Disease": false,
  "Upper GI symptoms": false,
  "ICU Stress Ulcer Prophylaxis": false,
  "Barrett's Esophagus": false,
  "Chronic NSAID use with bleeding risk": false,
  "Severe esophagitis": false,
  "Documented history of bleeding GI ulcer": false,
  "Reasoning": "The patient's notes do not mention any of these conditions. The patient is being treated for cancer and has symptoms such as nausea, vomiting, and abdominal pain, but these are attributed to a small bowel obstruction (SBO) rather than any of the listed conditions. The patient is also being treated for pain and symptom management, but there is no mention of any of the listed conditions."
}
```

However, the patient is on famotidine 20 mg Oral Daily PRN, which is a medication used to treat conditions such as GERD and peptic ulcer disease. But there is no clear indication in the notes that the patient has been diagnosed with any of these condi